Design patterns help you learn from others’ successes instead of your own failures. Design Pattern: how to solve an entire class of similar problem. This usually only appears after applying a standard design a number of times, and then seeing a common pattern throughout these applications.

L'objectif des *creational design patterns* est de fournir un certain nombre de mécanismes de création d'objets permettant d'isoler les détails relatifs à la création des objets du code qui les utilise. Le code client étant indépendant des implémentations concrètes des objets qu'il peut utiliser, il n'a par exemple pas à être modifié lorsqu'un nouveau type d'objet doit être introduit.

Les *structural design patterns* s'intéressent à la façon dont les différents objets composant le programme sont assemblés, connectés, mis en relation, l'idée étant que des changements puissent être apportés au système sans que ces relations entre objets aient à être modifiées.

Les *behavioral design patterns* s'attachent à l'assignation des responsabilités et donc des comportements entre les objets. Là encore, l'objectif est de découpler un code client réprésentant une logique de haut-niveau des implémentations concrètes de chacune de ses sous-logiques.

A ces trois catégories très connues du fait de leur description dans le GoF, on peut mentionner d'autres familles de *design patterns* qui sont soit davantage attachés à un paradigme (Ex: les *functionnal patterns* comme le Generator) soit à des situations/problématiques spécifiques: *architectural patterns* (Ex: *Inversion of Control*, *Model-View-Controller*, etc.), *concurrency patterns*, *cloud/distributed patterns*, etc.


Creational:
* Factory method: Encapsulating object creation
* Abstract factory: Encapsulating object creation
* Prototype
* Singleton
* Builder

Structural:
* Adapter: Changing the interface
* Bridge 
* Composite
* Decorator: Dynamic type selection
* Facade: Changing the interface
* Flightweight
* Proxy: Fronting for an implementation

Behavioral:
* Chain of Responsability: Function object
* Command: Function object
* Iterator: Decoupling algorithms from containers
* Mediator
* Memento
* Observer
* State: Fronting for an implementation (StateMachine ?)
* Strategy: Function object
* Template: Building application frameworks
* Visitor

An iterator allows you to hide the particular implementation of the container as you’re stepping through and selecting the elements one by one. Thus, you can write generic code that performs an operation on all of the elements in a sequence without regard to the way that sequence is built. Thus your generic code can be used with any object that can produce an iterator.

# *Creational design patterns*
L'objectif des *creational design patterns* est de fournir un certain nombre de mécanismes de création d'objets permettant de maintenir la flexibilité et le réemploi du code. 

In [1]:
import abc

# The Creator's primary responsability is not creating products. It usually contains some core business logic 
# relying on the abstract Product interface. This logic can be overriden by the concrete Creator subclasses.
class Creator(abc.ABC):
    @abc.abstractmethod
    def factory_method(self):
        pass
    
    def some_operation(self):
        product = self.factory_method()
        result = product.operation()
        return result
    
class ConcreteCreator1(Creator):
    def factory_method(self):
        return ConcreteProduct1()
    
class ConcreteCreator2(Creator):
    def factory_method(self):
        return ConcreteProduct2()
    
class Product(abc.ABC):
    @abc.abstractmethod
    def operation(self):
        pass
    
class ConcreteProduct1(Product):
    def operation(self):
        return 'Performed operation with product 1 specific logic'
    
class ConcreteProduct2(Product):
    def operation(self):
        return 'Performed operation with product 2 specific logic'

In [2]:
# The client code works with an instance of a concrete creator (not directly but rather through the 
# abstrat Creator interface). The concrete creator instance is a dependency of the client code which gets it
# by injection.
def client_code(creator):
    print(creator.some_operation())
    
client_code(creator=ConcreteCreator1())
client_code(creator=ConcreteCreator2())

Performed operation with product 1 specific logic
Performed operation with product 2 specific logic


##### Dynamic registry for factories : 
https://medium.com/@geoffreykoh/implementing-the-factory-pattern-via-dynamic-registry-and-python-decorators-479fc1537bbe

Le Creator est ici (et de façon un peu trompeuse vu son nom) seulement utilé par le code client pour exécuter 
de la business logic. La factory_method n'est pas directement appelée par le code client. Le Creator refactorise 
ici:
* La logique de création du Product dans la factory method
* Une (ou plusieurs ?) business logic s'appuyant sur l'interface Product

L'idée de la factory est que le code client ne fasse plus directement appel au constructeur des objets qu'il utilise car ceux-ci créent un couplage entre le code client et les classes concretes qu'il utilise ce qui à terme peut se traduire par un code client parsemé de conditions et dont le comportement dépend finalement de la classe concrète utilisée. Le refactoring consiste alors à faire s'appuyer le code client sur une interface (ici Creator mais pourquoi pas directement Product ?), la construction des objets étant réalisée dans la factory method. Le code client s'appuyant désormais sur une interface, son comportement ne dépend plus de la classe concrète utilisée: l'interface permet le découplage entre les deux, les logiques propres à chaque classe étant refactorisée dans les impémentations concrètes de l'interface (Creator ou Product).

La factory refactorise la logique qui préside au choix de l'implémentation concrète dont l'instance est finalement retournée (les if avec les appels aux constructeurs).

Ici on a l'impression qu'on a une implémentation concrète de Creator par implémentation concrète de Product. Où est donc remontée la logique permettant de choisir le bon Creator ? Le choix du Creator n'est pas forcément lié au choix de un Product. La factory method d'un Creator donné peut permettre la création de différentes implémentations de Product. Ex: Creator SeaLogistics qui peut retourner BigBoat, SmallBoat vs LandLogistics qui peut retourner Car, Truck. BigBoat, SmallBoat, Car, Truck implémantant une interface commune Transportation sur 
laquelle s'appuie le code client et/ou de la business logic contenue dans le Creator.

Factory is not a design pattern by itself; rather, it's a concept that serves as a basis for
several design patterns such as Factory Method and Abstract Factory.

Organisation de l'application en layers. Chacune étant plus abstraite que celle du dessous. Principe d'abstraction ?

Coder contre une interface et pas une implémentation : le code client n'a plus à faire d'hypothèses sur les systèmes sur lesquels il s'appuie, il s'appuie à la place sur des contrats.

You avoid tight coupling between concrete products and client code.
 Single Responsibility Principle. You can extract the product creation code into one place, making the code easier to support.
 Open/Closed Principle. You can introduce new variants of products without breaking existing client code.


Frontières du pattern: comment est choisie (voire passée) la bonne classe concrète ? Sur quelle base se fait le switch/case. La conf?

You can extract the product creation code into one place (peut inclure la logique consistant à vérifier si l'objet existe déjà - si sa création coûte cher par exemple -, à ajouter l'objet créer à un object pool, etc.):
* Le code client a-t-il besoin de plusieurs objets dont on doit assurer qu'ils sont compatibles entre eux ? Réponse: Abstract Factory
* Le code client a-t-il besoin d'un objet relativement complexe à construire et/ou pouvant se décliner en de nombreuses variantes (différents types d'objets classables sur plusieurs dimensions) ? Réponse: Builder
* Le code client fait appel à un code dépendant lui-même que du type de l'objet à construire (objets possibles variants suivant une seule dimension => gérable en héritage: chaque implémentation sous-classe une base classe) ? Réponse: Factory method. On regroupe code dépendant de l'objet (business logic) et création de l'objet dans un même Creator object. Les deux logiques sont isolées dans des méthodes séparées, la création de l'objet étant dans la factory method qui est réimplémentée par chaque implémentation de l'interface Creator. Le code client ne travaille que sur l'interface Creator, le choix de l'objet finalement utilisé dans la business logic étant fait lors du choix par le code client de l'implémentation concrète du Creator qu'il compte utiliser.
* Le code client peut-il simplement se contenter d'une copie d'un objet existant / Veut-on exploiter la possibilité de cloner un objet existant ? Réponse: Prototype (il s'agit se s'assurer que les objets à copier implémentent bien ce pattern, i.e.: mettent à disposition une méthode clone/copy dans leur interface).
* Veut-on s'assurer qu'il n'existe qu'une seule instance de l'objet à créer ? Réponse: Singleton


On veut découpler le code client de ses dépendances: on le refacotorise et le variabilise avec une interface qui devient de facto un paramètre du code. Le code client ne dépend plus que d'une interface. La dépendance à passer au code client est la classe concrète. Comment celà est-il fait ?
* Création: le code client se crée ses objets à l'aide d'un creationnal pattern:
    * Si factory method: Le code client est en fait encapsulé dans un objet Creator. Choisir la bonne classe concrète utilisée dans le code client correspond à choisir la bonne implémentation du Creator => Fait en amont.
    * Si abstract factory: Il faut injecter à l'objet utilisateur la bonne implémentation de la factory.
    * Prototype: le code client se crée un clone d'un objet existant: comment l'obtient-il ? On lui injecte le clone ? ou au moins l'objet et il se le clone lui-même (sans intérêt)
    * Singleton: Injecté ? Forme de variable globale / récupéré via méthode de construction statique: pas besoin de l'injecter.
    * Builder: Le code client doit a minima récupérer/instancier la bonne implémentation concrète du builder et décider de la séquence de construction à exécuter (soit en appelant les méthodes du builder directement, soit en appelant une méthode particulière d'un Director object qu'il aura fallu préalablement récupérer). 
* Injection: on passe le bon objet au code client, la sélection étant faite en amont.


Builder: Répond à la problématique de la construction d'objets complexes. Si la construction / l'initialisation de l'objet requiert la construction / l'initialisation de nombreux composants pouvant chacun requérir un certain nombre de paramètres, cela va certainement se manifester par un très gros et complexe constructeur pourvu d'un nombre très important de paramètres dont seule une fraction sera régulièrement utilisée. Le Builder vient répondre à cette problématique. Il s'agit de déléguer à une classe la construction de l'objet, celle-ci refactorisant l'énorme constructeur en un ensemble de méthodes permettant de construire les composants individuels de l'objet à construire. L'objet à construire qui est pensé comme l'association (aggrégation ou composition - qui revient à déléguer à d'autres objets) de ses composants est d'abord initialisé "à vide" à la création du builder, puis lui est ajouté un composant par appel à une des méthodes du builder avant d'être finalement récupéré complet par le code client. La séquence des étapes de constructions peut être directement réalisée dans le code client ou on peut au contraire rassembler différentes séquences permettant de construire différentes variantes de l'objet (construction routines) à l'intérieur d'un objet appelé Director qui est utilisé par le code client qui aura pris soin de lui passer la bonne implémentation du Builder. L'utilisateur du Builder qu'il soit le code client ou le Director s'appuie en effet sur une interface et la possibilité d'avoir plusieurs implémentation de builders permet de créer (et de gérer harmonieusment) différentes *flavors* d'un même type d'objet (défini par une séquence de construction particulière). Chaque méthode du Director correspond à une séquence de construction particulière et donc à un type, à une variante d'objet particulière. L'utilisation d'un Director et d'un Builder permet ainsi d'isoler les séquences de construction (construction routines) d'un côté (dans le Director) et les implémentation concrète de chacune des étapes de construction de l'autre (dans le Builder).

Dans tous les cas, il faut obligatoirement définir une interface pour le builder, le code client/le Director s'appuyant dessus. Eventuellement aussi pour le Director si utilisé à plusieurs endroits. 

Le pattern Builder vient répondre à la problématique de la création 1) d'objets complexes pouvant (ou non) 2) être de différents types, chaque sous-type pouvant (ou non) 3) présenter différentes variantes. La gestion naïve de cette situation par héritage (chaque objet correspond à une sous-classe d'un même base object - lui même implémentant une interface) n'est pas possible du fait du trop grand nombre de classe que cela générerait. Une réponse serait alors de penser l'objet à construire comme l'association de composants indépendants et de rassembler la logique de construction et la combinaison des choix possible dans un même constructeur. Cela n'est pas non plus souhaitable: le constructeur serait vraisemblablement très complexe et une combinatoire élevée se traduisant par un nombre trop important d'arguments pour celui-ci. Le pattern Builder vient répondre à ces problématiques. Chacune des méthodes du builder correspond à l'implémentation concrète de la construction d'une variante d'un composant de l'objet final. La gestion des différents types d'objet est gérée au niveau de la combinaison des opérations de construction au niveau du Director: chaque méthode du Director permet de construire un type d'objet particulier définit par la séquence des étapes de construction (méthodes du Builder ) utilisée. La gestion des différentes variantes est gérée au niveau du Builder: à chaque variante correspond une implémentation concrète du Builder. Le Director ne s'appuie que sur une interface, au client de lui passer le Builder concret adapté à la variante de l'objet qu'on souhaite construire.

Différents types d'objet au sens qu'on peut les classer suivant deux dimensions indépendantes de variabilité: une est gérée par le Director, l'autre par le Builder.

Remarque: Inheritance drawbacks: à chaque nouvelle variété doit correspondre une sous-classe d'une base class. Approche inadaptée et contre-productive en cas de forte combinatoire. Le pattern Builder est une illustration du passage d'un refactoring s'appuyant sur l'héritage à un refactoring s'appuyant sur l'association comme réponse au problème d'une combinatoire élevée. Dans le cas de l'héritage, l'objet peut faire le travail lui même puisqu'il hérite le comportement de sa classe parente, dans le cas de l'association, le travail est délégué à un autre objet.

Prototype: L'idée de ce pattern est de sortir du code client la logique dédiée à la copie de l'objet. C'est à l'objet de mettre à disposition du client une méthode (intégrée à l'interface sur laquelle s'appuie le code client) permettant de dupliquer (on dit aussi cloner) une instance particulière. Plusieurs raisons:
* Laisser la logique de duplication dans le code client a de fortes chances de créer un couplage en celui-ci et des implémentations concrètes (mal dit) de la classe à laquelle appartient l'objet à dupliquer
* L'opération de clonage qui revient à copier les valeurs prises par l'ensemble des membres de la classe peut ne pas être possible: 
    * Certains membres peuvent être privés et non accessibles au code client.
    * Le code client peut ne connaître que l'interface (publique) implémentée par l'objet à cloner: il n'a pas connaissance de l'ensemble des membres composant la classe.

La logique de clonage peut être assez compliquée: linked objects, recursive or circular dependencies, etc.
Le pattern Prototype conduit à définir une interface permettant de définir un objet clonable. Python propose déjà une Prototype interface via sa fonction copy.copy: afin de rendre son object clonable avec cette méthode, on a juste besoin de réimplémenter les méthodes __copy__ et __deepcopy__.

Ce pattern répond à la problématique de privilégier la copie d'objets existants pour créer les objets dont on a besoin. L'idée peut par exemple consister à s'appuyer sur un registre (à implémenter) rassemblant un petit nombre d'objets pré-construits (et dont la construction coûte potentiellement cher) et de ne faire que le requêter pour en copier les objets (avant d'éventuellement en modifier quelques attributs ensuite ?). 

Singleton: Le pattern singleton vient répondre à deux problèmes: 
* Assurer que la classe ne peut être instanciée qu'une seule fois
* Fournir un accès global à l'objet instancié (comme une variable globale mais en plus safe: risque moins d'être écrasée).
Le Singleton s'implémente en rendant le constructeur de la classe privée et en implémentant une méthode statique qui contient la logique permettant de ne retourner que l'instance déjà créée. Le fait que la méthode soit statique permet l'accès global.
Attention: Suivant les cas d'usage, le pattern Singleton peut être vu comme un anti-pattern (si mutable/pas que read-only ?/objet trop gros/qui en sait trop sur les autres, etc.) ou au moins exiger certaines précautions (ex: multithreading).

# Structural patterns
Structural patterns explain how to assemble objects and classes into larger structures while keeping these structures flexible and efficient.

Bridge 
L'idée de ce pattern est de répondre à la problématique posée par l'héritage: la multiplication exponentielle des sous-classes avec l'augmentation de la variété des objets. L'idée du Brige va consister à créer à partir de deux dimensions indépendantes de variabilité des objets, deux hiérarchies distinctes de classes. Les deux hiérarchies sont créées de façon à ce qu'une des deux hiérarchies de classe gère des aspect de relativement haut-niveau et que le travail concret soit délégué aux objets de la seconde hiérarchie (ce qui demande également de définir une interface, celle sur laquelle s'appuie la couche de haut-niveau). On a donc ainsi créé deux couches dans l'application, une de haut niveau appelée abstraction et une de plus bas niveau appelée implémentation. Le lien (brigde) entre les deux hiérarchies de classes est maintenu par association: les objets de la couche d'abstraction comptent comme membre une instance de la couche d'implémentation. 

On peut ainsi se focaliser sur les logiques de haut-niveau dans la couche d'abstration et sur les détails d'implémentation dans la couche de plus bas-niveau, les deux pouvant être étendues indépendemment. Le pattern fait ainsi appel à l'association afin de répondre aux problèmes posés par l'héritage et de permettre de refactoriser une grosse classe en deux familles de logiques distinctes: des logiques de plus haut-niveau dans la couche dite d'abstraction et des logiques de plus bas-niveau auxquelles la couche d'abstraction délègue dans la couche dite d'implémentation. 

Le pattern Builder est un cas particulier de Bridge: comme le Bridge, le Builder vient répondre aux problèmes posés par l'héritage en recourrant à l'association. Dans le Builder, l'abstraction correspond au Director qui rassemble des logiques de relativement haut-niveau (les construction routines), les détails d'implémentation étant laissée aux classes Builder qui constituent l'implementation layer.

Decorator
L'idée du pattern Decorator est de pouvoir attacher des comportements additionnels à un objet en le passant à un wrapper object, le decorator.

Problématique: En recourant uniquement à l'héritage, soit les comportement s'héritent, soit s'ajoutent par surcharge des méthodes de la classe parente. Dans tous les cas cette approche pose vite problème: 
* La combinatoire des différents comportements possibles peut rapidement exploser, à chaque combinaison de comportements devant correspondre une sous-classe de la classe de référence.
* L'héritage multiple (remarque: mixins, traits ?) peut permettre de facilement combiner des comportements issus de différentes classes parentes. Toutefois de nombreux languages n'autorisent que l'héritage simple. 
* L'héritage est statique: le comportement d'un objet ne peut pas être altéré à l'exécution (runtime), tous les comportements désirés doivent exister et seul l'objet correspondant au comportement voulu est choisi à l'exécution. 

Comme d'autres patterns, le Decorator vient répondre aux problèmes posés par l'héritage en optant pour l'association: plutôt que d'hériter ou d'implémenter tous ses comportements, l'objet en délègue à d'autres objets. Le Decorateur est un objet wrapper qui implémente la même interface que l'objet à wrapper, ainsi du point de vue du code client, il n'a aucune différence entre utiliser l'objet wrappé et l'objet original. 

Design patterns : découpler le code client des implémentations concretes des objets qu'il utilise. Rendre les différents composants du programme les plus indépendants possible.

Inheritance vs delegation
Refactoring: the right level of abstraction?/dependecy injection or not - dependency management/conf management
Ex: State can be considered as an extension of Strategy. Both patterns are based on composition: they change the behavior of the context by delegating some work to helper objects. 

A pattern isn’t just a recipe for structuring your code in a specific way. It can also communicate to other developers the problem the pattern solves. => Expliciter le problème de chaque pattern

## Behavioral design patterns
Behavioral design patterns are concerned with algorithms and the assignment of responsibilities between objects.

## *Mixins* et *traits*

### *Mixins*
Un mixin (ou mix-in) s'entend comme une classe rassemblant uniquement des méthodes concrètes destinées à ajoutées des classes utilisatrices. Un mixin est une classe particulière dans le sens où elle n'est pas destinée à être la classe parente des classes qui récupèrent ses comportements. Il n'y a en théorie pas de *"is a"-relationship* pour un mixin. La relation qu'entretient une classe et son mixin n'est donc pas pensée comme de l'héritage, on préfère ainsi dire qu'une classe "inclut" un mixin plutôt qu'elle hérite d'un mixin. Ces considérations sont théoriques et peuvent ne pas apparaître aussi explicitement en pratique dans la syntaxe des languages supportant les mixins. Python par exemple supporte les mixins au sens où leur implémentation est rendue possible par le language mais pas explicitement implémentés par celui-ci (pas de *keywords*/*statements* dédiés). En Python, un mixin n'est qu'une classe à part-entière et le language supportant l'héritage multiple, les classes utilisatrices en récupèrent les comportements par héritage ce qui peut paraître contradictoire avec la définition donnée plus haut (d'autant que cela introduit une *"is-a" relationship*). Les mixins qui sont des classes comme les autres: le mixin est davantage un "rôle" particulier attaché à une classe. Le mixin n'est alors simplement qu'une classe dont le but n'est pas de définir de *"is-a"/"is-a-type-of" relationship*. Ruby en revanche implémente le concept de mixins via des objets appelés `Modules` que des classes récupèrent les comportements à l'aide du *keyword* `include`. On répète que l'héritage n'est normalement pas le mécanisme par lequel est implémenté le support de mixins: les comportements sont normalement ajoutés et non héritées (méthodes copiées dans le namespace de l'objet et non récupérées en remontant une chaîne d'héritage).

Les mixins permettent de refactoriser dans une classe un ensemble de comportements partagés par un ensemble de classes mais pour lequelles on ne peut concevoir une unique classe parente commune.

Les mixins permettent ainsi de construire des classes par composition: aux attributs et comportements hérités viennent s'ajouter ceux apportés par les différents mixins inclus. Les mixins sont ainsi souvent vus comme une alternative à l'héritage multiple qui n'est pas supporté par tous les languages.

Les mixins ne sont pas des interfaces au sens où:
1. Aucune des méthodes d'un mixin sont abstraites et aucune réimplémentation n'est exigée (voire n'a de sens),
2. Les mixins ne changent pas la nature de la classe: la relation d'une classe a son mixin n'est (en théorie) pas de l'héritage. A une interface (par définition abstraite) sont associées un ensemble d'implémentations concrètes qui ont une *"is a"-relationship* avec la première (relation d'héritage). Il n'y a pas de `AnimalMixIn` dont hériteraient des classes `Cat` et `Dog`.

On les désignent parfois cependant comme des *interfaces with implemented methods* au sens où tous les objets incluant le ou les mixins partagent les mêmes comportements. Les *mixins* partagent également avec les interfaces (ou plutôt les classes abstraites qui permettent de les implémenter) le fait qu'ils ne sont pas destinés à être instanciés car cela n'a au fond pas de sens.  

#### Les *mixins* en Python

La principale implémentation des *mixins* en Python s'appuie sur l'héritage multiple: il suffit de faire hériter à la classe à laquelle on souhaite ajouter des comportements le ou les mixins désirés.

```python
class RadioUserMixIn:
    def __init__(self):
        self.radio = Radio()
        
    def play_song_on_station(self, station):
        self.radio.set_station(station)
        self.radio.play_song()

class Car(Vehicle, RadioUserMixIn):
    pass

class Clock(Vehicle, RadioUserMixIn):
    pass
```

On peut églament aboutir au même résultat à la l'aide d'un décorateur de classe qui ajouterait des méthodes à la classe au moment de l'assignation. Ce procédé semble toutefois moins clair et sans doute même un peu déroutant, sans doute vaut-il mieux privilégier l'implémentation passant par l'héritage multiple même si cela ne respecte pas totalement l'esprit de la définition d'un *mixin*. 

### *Traits*
*Traits* et mixins semblent des notions assez proches: le *trait* se définit par exemple comme la représentation d'une *feature*, d'un aspect totalement orthogonal aux responsabilités d'un type concret (d'une classe particulière). Un *trait* permet ainsi en particulier de représenter des comportements partagés par des classes qui n'ont sinon rien en commun. Les *traits* semblent présentés comme des "mixins augmentés" notamment au sens qu'ils peuvent être composés via un certain nombre d'opérations pour créer de nouveaux *traits*, exigent une résolution explicite des conflits si une méthode du même nom est présente dans plusieurs *traits*, etc.. Scala par exemple propose explicitment des `Traits` (associés au *keyword* `extends`) qui lui permettent d'implémenter le *mixin behaviour*.

On voit parfois aussi les *traits* présentés comme l'implémentation du *mixin behaviour* qui est alors une *feature* du language. Il semble qu'il n'y ait pas de consensus sur une distinction claire et nette entre *traits* et *mixins*.

Plus largement, *mixins* et *traits* sont reliés à la théorie des types. 

## *Type systems*, *duck typing* et protocoles Python
### Systèmes de typage
Tous les languages de programmation incluent une forme de système de typage (*type system*) qui consiste en un ensemble de règles permettant d'assigner à chaque entité du programme (variables, expressions, fonctions, modules, etc.) une propriété appelée type. Le problème fondamental de la théorie des types (qui s'intéresse aux systèmes de typage) est de faire en sorte que les programmes aient un sens, une cohérence. L'action de typer, de donner un *data type* à une séquence de bits stockés quelque part en mémoire vient donner un sens, une signification à la variable ou à l'objet qu'ils représentent. Un programme assigne ainsi au moins un type à chaque valeur, une même valeur pouvant se trouver associée à différents (sous-)types. Le type est une information supplémentaire, une métadonnée, un identifiant associée à une valeur. En pratique on parle de *data type* pour le type d'une valeur, de classe (*class*) pour le type d'un objet et de sorte (?) (*kind*) pour le type d'un type (métatype) car un type peut aussi être associé à un type (en Python où tout est objet par exemple, une classe est un objet de type `type`).

Adjoindre un type à chaque objet revient à créer un ensemble de symboles, de signes qui vise à donner un sens et une cohérence au programme. L'ensemble de règles associé à un système de types permet de maintenir et de contrôler la cohérence que les types apportent au programme. On peut également citer deux autres avantages que procure le typage:
* Abstraction: Les types sont par essence un symbole, une abstraction qui permet au programmeur de se doter de représentation de plus haut niveau des entités de son programme et l'aide à formuler et à expliciter les interfaces entre différents composants du programme. Les types contribuent donc à formuler des abstractions utiles et à augmenter la modularité du code. 
* Documentation: Intrinsèquement expressifs car apportant une signification aux différentes entités d'un programme, les types contribuent à documenter le code et à le rendre intelligible. 

Au fond, les types ne font que formaliser et implémenter des catégories utilisées par le programmeur pour décrire les relations entre différents objets. **Les types permettent de définir des catégories d'entités compatibles ou équivalentes** et contribuent à nous assurer que deux entités peuvent travailler ensemble. Les types définissent ainsi des interfaces entre différents composants d'un programme et en permettant de contrôler si ces différents éléments on été agencés de façon cohérente, permettent d'éviter des bugs.

Les différents *type systems* sont classés suivant trois axes décrivant leurs principales caractéristiques:
 * Statique (*static*) vs. dynamique (*dynamic*)
 * Nominatif (*nominal*) vs. structurel (*structural* ou *property-based*)
 * Manifeste (*manifest*) vs. inférré (*inferred*)

#### Systèmes statiques vs. dynamiques
Le premier axe fait référence au moment où a lieu le *type checking*. Dans un language statiquement typé, le *type checking* a lieu à la compilation (*at compile time*) alors qu'il a lieu à l'exécution (*at run time*) dans les languages dynamiquement typés. Les deux ne sont pas forcément mutuellement exclusifs, certains languages peuvent proposer une combinaison des deux. Une des conséquences visibles pour le programmeur est par exemple que dans un language dynamiquement typé, une même variable peut changer de type au cours de son cycle de vie dans le programme alors que c'est souvent impossible dans les languages statiquement typés où on ne peut assigner à une variable que des entités correspondant au type qu'elle a déclaré.

#### Systèmes nominatifs vs. structurels
Le second axe fait référence au critère permettant d'affirmer que deux types sont équivalents ou non. Chaque language se dote d'un ensemble de critère qui lui est propre pour déterminer si deux expressions sont "du même type". Ces règles varient largement entre deux pôles extrêmes constitués par les systèmes nominatifs et structurels.

De nombreux languages implémentent la notion de sous-typage qui 1) doit être définie et 2) vient complexifier la notion d'équivalence entre deux types, le sous-typage sous-entendant une notion de substituabilité puisque si S est un sous-type de T alors S peut être utilisé dans un contexte où T serait normalement attendu, la réciproque n'était pas forcément vraie.

Remarque: Le sous-typage est une des principales formes de polymorphisme, notion elle-même très fortement liée au typage puisque qu'elle se définit comme le fait de fournir une même interface à des entités de types différents. 

#### Systèmes nominatifs
En typage nominatif: 
* Compatibilité: Deux variables ont un type compatible si et seulement si leurs déclarations utilisent le même nom de type (à un alias de type près).
* Sous-typage: Un type est un sous-type d'un autre si et seulement si il est explicitement déclaré comme tel dans sa définition. Dans le cas de la POO, le type d'un objet correspond à sa classe et une relation d'héritage est considérée comme une déclaration explicite de relation de sous-typage.

Retenir que les systèmes nominatif se basent sur des déclarations explicites (noms, relations explicites) pour déterminer l'équivalence ou la compatibilité de types.

Remarque: En Python, évaluer le sous-typage en ne s'appuyant que sur la *class hierarchy* se ferait à l'aide des *built-in functions* `isinstance` et `issubclass`. C'est sans doute parce que Python n'utilise pas typage nominatif que leur usage peut être qualifié, ou au moins apparaître *unpythonic*. 

#### Systèmes structurels
En typage structurel la compatibilité et l'équivalence de type est déterminée par la structure du type (i.e.: la structure utilisée pour décrire la valeur typée) et non par des déclarations explicites comme dans les systèmes nominatifs. Ainsi:
* Compatibilité: Deux types sont considérés comme compatibles si pour chaque fonctionnalité (*feature*) du second on trouve une fonctionnalité équivalente dans le premier. La notion d'équivalence des fonctionnalité varie suivant les languages. On voit par ailleurs que cette relation n'est pas symétrique (tous les fonctionnalités du premier peuvent ne pas être présentes dans le second). Deux types sont ainsi définis comme identiques si et seulement si chacun est compatible avec l'autre.
* Sous-typage: Un type est un sous-type d'un autre si et seulement si il contient toutes les fonctionnalités du type de base ou de l'un de ses sous-types. Le sous-type peut en particulier contenir plus de fonctionnalités que son type de base.

Remarques:
* Compatibilité et identité semblent être confondues en typage nominatif, pas en typage structurel.
* Equivalence de fonctionnalités: En ce qui concerne les méthodes, avoir le même nom peut ne pas suffire. L'équivalence impose souvent des contraintes sur la signature.

#### Compromis flexibilité - *type safety*
Les languages utilisant principalement un système nominatif imposent le plus souvent que lorsqu'un type est déclaré comme le sous-type d'un autre, les deux types soient également structurellement compatibles. Les systèmes nominatifs sont considérés comme plus sécurisant (*type safe*) car il permettent de mieux prévenir les équivalences de type accidentelles (*accidental type equivalences*) typiques des systèmes structurels. Dans ces derniers, il est en effet possible de déclarer compatibles deux types créés pour des objectifs très différents s'ils se trouvent avoir "accidentellement" des structures compatibles. Un compromis apparaît ainsi entre système nominatifs apportant de la *type safety* et les systèmes structurels qui donnent de la flexibilité au programmeur. On peut donner comme expemple de la flexibilité autorisée par les systèmes structurels le fait qu'ils autorisent la définition d'un super-type à un type existant T sans avoir à modifier la définition de T (en système nominatif, la définition de T devrait être adaptée pour le déclarer sous-type du super-type créé). Cela autorise dans les systèmes structurels la création de super-types, d'interfaces *ad hoc* aussi appelées protocoles (*protocols*).

Remarque: La *type safety* ne consiste pas simplement à prévenir les *accidental type equivalence*. Il s'agit plus largement de la capacité qu'à un language à décourager voire prévenir les *type errors*. Un système de type visant plus largement à donner du sens au programe, il s'agit donc de la capacité du language à détecter du code incohérent et probablement incorrect.

### Typage manifeste vs. inférré
Le typage manifeste se caractérise par le simple fait qu'il est requis de la part du programmeur d'identifier explicitement le type dans le code de déclaration de chacune des variables. A l'opposé, l'*inferred typing* désigne la détection automatique du *data type* d'une expression.  

### *Duck typing*, typage structurel et protocoles
Le *duck typing* est un autre système de types **distinct mais semblable** au système struturel. Il en diffère sur deux points dans le sens où seule **la partie de la structure** accédée **à l'exécution** est utilisée pour contrôler la compatibilité, là où un système structurel utiliserait l'ensemble de la structure potentiellement à la compilation. Le *duck typing* diffère plus radicalement des systèmes nominatifs dans le sens où le nom peut ne pas être du tout utilisé pour contrôler l'identité ou la dépendence d'un type avec un autre. Dit autrement: on ne contrôle pas le type de l'objet (nom ou structure) avant de l'utiliser (ex: de le passer à une fonction), on s'assure juste au moment de son utilisation qu'il implémente juste ce dont on a besoin. Le *duck typing* n'est logiquement implémentée que par certains languages dynamiquement typés. Python par exemple est *dynamically and duck-typed*. C'est un système populaire car d'une grande flexibilité même s'il partage avec les systèmes structurels la possibilité d'accidents.

Remarques: 
* Ce système tire son nom de l'expression "*If it walks like a duck, and quacks like a duck, then it is a duck*" appelé aussi *duck test*.
* On voit parfois de typage structurel désigné comme du *static duck typing*. 

Un des avantages du *duck typing* (et du typage structurel) est de permettre la définition d'interfaces *ad hoc* relativement flexibles. Imaginons qu'une *third-party library* mette à disposition une classe non modifiable par l'utilisateur. Dans un système nominatif, ce dernier ne peut utiliser la classe avec une interface que la librairie ne connait pas même si dans les faits la classe l'implémente: il faudrait que la classe dise explicitement qu'elle implémente cette interface cependant l'utilisateur ne peut pas aller modifier la classe. La solution consiste alors le plus souvent à utiliser la classe via un Adapter (*design pattern*). Le *duck typing* à l'inverse autoriserait une utilisation directe de la classe puisque l'interface (constituée simplement des méthodes finalement appelées) est satisfaite. On remarque que les interfaces définies à l'aide du *duck typing* sont implicites: elles ne sont pas explicités à l'aide de classes abstraites par exemple. Sans doute pour les distinguer de ces "interfaces explicites", on s'y réfère avec le terme de "protocole" (*protocol*).

### Les protocoles en Python
Etant *duck-typed*, Python utilise des protocoles dont on peut citer les plus connus: 
* `Sized` qui demande à l'objet d'implémenter une méthode `__len__` afin de pouvoir être passé à la *built-in function* `len`.
* `Iterable` qui demande à l'objet d'implémenter une méthode `__iter__` retournant un objet `Iterator` afin de pouvoir être passé à la *built-in function* `iter`. On remarque que la définition ainsi faite d'un objet "itérable" n'implique pas qu'il soit indexable, qu'il ait une longueur, ni même qu'il soit fini. Respecter ce protocole permet également d'être directement utilisé dans une boucle `for` qui va commencer par récupérer l'`Iterator` associé à notre objet pour ensuite le traverser en utilisant sa méthode `__next__` jusqu'à ce que soit levée l'exception `StopIteration`.
* `Iterator` qui demande à l'objet d'implémenter le protocole `Iterable` et une méthode `__next__` afin de pouvoir être passé à la *built-in function* `next`. La méthode `next` doit lever l'exception `StopIteration` quand il n'y a plus d'éléments à retourner.
* `Container` qui demande à l'objet d'implémenter une méthode `__contains__` retournant un booléen. L'objet peut alors être utilisé avec l'opérateur `in`.
* `Collection` qui se définit comme l'intersection des protocoles `Sized`, `Iterable` et `Container`. Voir notamment le *built-in package* [`collections.abc`](https://docs.python.org/fr/3/library/collections.abc.html) qui explicite à l'aide de classe abstraites (qui du même coup constituent des types génériques) un certain nombre de protocoles utilisés par le language. Ex: `Collection`, `Sequence`, `Mapping`, `Generator`, etc.
* Les *context managers* constituent un protocole en demandant implicitement à leurs implémentations de posséder deux méthodes `__enter__` et `__exit__`, la dernière devant également posséder une signature particulière.

Remarque: L'essentiel des *magic methods* de Python s'appuient en fait sur un protocole. 

### Remarque - Le *monkey patching*
Dans les languages dynamiques, le *monkey patching* désigne le fait de modifier dynamiquement la définition d'une classe (ex: méthode, attribut) ou d'un module à l'exécution en général pour modifier ou augmenter du *third-party code* ou de contourner un bug. Cette pratique souvent à vocation temporaire est largement découragée pour du code destiné à la production du fait qu'il est souvent possible de produire le même effet avec un effort de design et que son utilisation est associée à de nombreux risques parmi lesquels on compte: 
* Problèmes d'*upgrade* si les hypothèses que fait le *patch* sur l'objet patché ne sont plus vraies. 
* Risque d'introduire des incompatibilités.
* Création d'un écart entre le code source présent sur le disque et le comportement observé à l'exécution potentiellement déroutant pour des gens ignorants de l'existence du patch.

Le terme proviendrait de l'expression *“'monkeying about' with the code”* (= *messing with it*).

#### TODO
* Caler mixin/traits quelque part. 
* Introduire les designs patterns

## Principes de la programmation orienté-objet
La programmation orienté-objet (POO) consiste à représenter les entités du problème à résoudre par des objets, la résolution impliquant l'interaction de ces objets entre eux. Un objet est une structure rassemblant les membres le définissant parmi lesquels on distingue des données (les attributs qui définissent l'état (*state*) de l'objet) et des comportements (les méthodes), les secondes pouvant agir sur ou modifier les premières. 

La plupart des languages de POO sont *class-based* (par opposition à *prototype-based*), la classe servant à spécifier l'ensemble des données et comportements disponibles pour des objets de même type (classe). Un objet est aussi appelé instance de classe. On peut établir une nouvelle distinction entre attributs/méthodes de classe (indépendants de l'instance et donc possédés l'ensemble de celles-ci) et d'instance.

Au delà des objets eux-mêmes, on dit souvent que la POO s'appuie sur quatre piliers:

### Abstraction
Les objets doivent interagir via des interfaces abstraites (implémentée via des classe dites abstraites dont héritent chacune des implémentations concrètes) permettant à chaque côté de ne pas avoir à connaître ou à dependre du fonctionnement de l'autre. L'utilisation d'une interface permet ainsi d'abaisser le couplage entre les deux parties du codes (voir aussi principe D des SOLID *principles*). Le concept d'abstraction peut aussi désigner le fait de dissimuler les logiques propres à l'objet dans des membres privés, l'objet ne donnant à voir à l'extérieur qu'une interface publique de plus haut niveau abstrayant son fonctionnement.

On trouve parfois dans les principes de programmation un "principe d'abstraction". Sans être ni défini formellement, ni un absolu, ce principe peut désigner plusieurs choses dont la définition et l'utilisation d'interfaces afin de réduire le couplage entre différentes parties du code, abstraire le comportement de certains systèmes en encapsulant leur logique dans des objets dédiés (favorisant ainsi la lisibilité et applicant la *separation of concerns*) ou encore l'exploitation du polymorphisme pour réduire la duplication du code et/ou *separate the concerns*. Dans tous les cas, ce "principe" est mis en avant comme devant toujours être dans la tête du programmeur, son application ne pouvant apporter que des bénéfices.

### Encapsulation
L'encapsulation désigne le concept de regrouper dans un objet un sous ensemble des fonctionnalités du système, c'est-à-dire un ensemble de données avec des méthodes permettant de les manipuler. Ce regroupement s'accompagne souvent de restriction d'accès aux membres de l'objet qui sont redus accessible via une interface publique explicitement définie. La restriction d'accès (appelée aussi masquage, ou *data hiding*) permet principalement trois choses:
* Elle permet de préserver les données d'une mauvaise utilisation par un code extérieur à l'objet (sécurité),
* Elle permet de garantir l'intégrité des données: l'utilisateur peut ne pas pouvoir modifier directement les données, facilité d'implémenter des règles de validation des valeurs à assigner à un attribut, etc.
* La restriction d'accès fait de l'objet une boite noire et constitue de ce fait une abstraction. Les membres de l'objet accessibles de l'extérieur en constitue *de facto* l'interface publique et permet au client de pouvoir faire abstraction du fonctionnement interne de l'objet. Celui-ci peut d'ailleurs changer sans causer de problème pourvu que le fonctionnement des méthodes publiques reste le même: l'encapsulation participe du découplage de différentes parties du code.

La limitation d'accès peut être implémentée par des *keywords* spéciaux comme les modificateurs de portée en Java (`private`, `protected` et `public`). De tels *keywords* n'existent pas en Python où tout est public. Les éléments privés d'un objet et ne devant *a priori* jamais être utilisés directement par un client sont par convention signalés en préfixant leur nom d'un ou deux *undescores*, la seconde méthode (appelée *name mangling*) rendant l'accès plus difficile mais pas impossible. Les *properties* (qui jouent le rôle de plus explicites *setters* et *getters* dans d'autres languages) constituent le mécanisme le plus approprié pour ce qui est du contrôle de l'accès aux attributs (pouvant aller jusqu'à les rendre *de facto* privés). 

### Héritage
L'héritage est un mécanisme proposé par la plupart des languages de POO (sauf par exemple Go) permettant de définir des hiérarchies entre différents types / classes (la classe étant le type d'un objet), entre classes-parentes et classes-filles (relations "*is-a-type-of*"). La plupart des languages (ex: Java) n'autorisent que l'héritage simple: une classe ne peut avoir qu'une seule classe parente. D'autres comme Python implémentent l'héritage multiple mais au prix de la définition de règles potentiellement complexes permettant de remonter la chaîne d'héritage.

Les classes-filles héritant des membres de leurs classes parentes et pouvant également totalement les surcharger/réimplémenter (*overriding*), l'héritage contribue à limiter la duplication du code en refactorisant le code partagé dans celui de la classe-parente et à appliquer la *separation of concerns* en faisant de chaque classe-fille une spécialisation particulière de la classe parente. 

La notion d'interface si utile pour diminuer le couplage entre différentes parties du code est elle aussi implémentée à l'aide de l'héritage. L'interface est souvent définie par une classe dite abstraire dont héritent toutes les implémentations concrètes, chacune encapsulant la logique propre à son sous-type. Dans les languages à héritage simple, une classe ne peut avoir qu'une seule classe parente mais implémenter plusieurs interfaces.

Remarques: 
* Suivant les languages, la classe abstraite servant à définir l'interface peut également refactoriser du code en autorisant certains de ses membres à posséder implémentation concrète (qui peut aussi faire office de comportement par défaut). C'est par exemple le cas de Python.
* Attention: L'héritage n'est pas un remède miracle au couplage en composants, il peut en introduire lui-même si par exemple les classes filles dépendent des détails d'implementation de leur classe parente. L'héritage introduit également un couplage évident: tout changement dans la classe parente impacte le comportement des classes filles.

#### Héritage vs. association
Du point de vue de l'assignation des responsabilités entre les objets, le programmeur doit arbitrer entre deux alternatives: l'héritage ou l'assocation, faire soi-même quitte à utiliser une méthode de ses parents ou déléguer à un autre objet. A chacun des deux mécanismes correspond un type de relation permettant d'organiser les objets: "*is-a-type-of*" pour l'héritage, "*has-a*" pour l'association. Insistons sur le fait que **l'association correspond à une délégation**. On distingue deux sous-types d'association: la composition et l'agrégation, le cycle de vie du contenu étant lié à celui du contenant dans le premier cas mais pas dans le second. L'héritage n'est en effet pas la panacée pour ce qui est de l'organisation des relations entre les objets et de nombreux *design patterns* s'appuient sur l'association pour palier aux limitations de l'héritage. On peut entre autre citer:
* La combinatoire des différents comportements possibles peut rapidement exploser, à chaque combinaison de comportements devant correspondre une sous-classe de la classe de référence.
* L'héritage multiple peut permettre de facilement combiner des comportements issus de différentes classes parentes. Toutefois de nombreux languages n'autorisent que l'héritage simple. 
* L'héritage est statique: le comportement d'un objet ne peut pas être altéré à l'exécution (*runtime*), tous les comportements désirés doivent exister *ex ante* et l'objet correspondant au comportement voulu est choisi à l'exécution. 

Quand doit-on opter pour l'héritage plutôt que l'association et inversement ? Partons de l'association. Imaginons un cas dans lequel une classe se retrouve à finalement déléguer à l'ensemble des méthodes publiques d'un même objet, on peut alors avoir un intérêt à changer la relation entre les deux objets en une relation d'héritage. On perdrait toutefois la **flexibilité de l'association** qui autorise différentes implémentations de l'objet auquel on délègue (on reconnait le *design pattern* Strategy).

A l'inverse, il y a des situations où l'association serait plus appropriée que l'héritage. Partons d'une situation où l'héritage est mal utilisé, par exemple le principe de substitution de Liskov est violé:
* Classe fille et classe parente ne sont pas substituables ou,
* La classe fille n'utilise qu'une partie des méthodes publiques de sa classe parente.
Dans ces cas là, il est justifié de passer d'une relation d'héritage à une relation d'association, l'ancienne classe fille déléguant à son ancienne parente.  

Certaines personnes diront que l'utilisation de l'héritage est rarement justifiée et que l'utilisation de l'association (et de l'injection de dépendances), d'interfaces et de *final base classes* suffisent. Dans cette vision, l'héritage est vu comme souvent trop rigide et contribuant à obscurcir les relations entre objets dans le programme.

### Polymorphisme
Le polymorphisme désigne le simple fait de fournir une même interface à des entités de types différents. Il désigne plus concrètement la capacité du code (notamment des fonctions et des classes) à agir sur des valeurs de types différents et en particulier la capacité de *data structures* à contenir des éléments de types différents. C'est une notion de théorie des types qui est plus larges que la POO puisqu'elle se rencontre sous des formes potentiellement différentes dans d'autres paradigmes comme la programmation fonctionnelle.

Dans un language dont le *type system* autorise le polymorphisme par exemple, un programmeur n'a à implémenter une structure telle qu'un *array* une seule fois et non pas une fois pour chaque type d'objets pour lesquels il sera utilisé (polymorphisme paramétrique). Le polymorphisme favorise ici grandement la réemployabilité du code / réduit la duplication de code. Un autre exemple: dans le cas du polymorphisme dit *ad hoc* on s'appuie sur le type de l'objet pour savoir à quelle méthode/fonction concrète il est finalement passé. On s'appuie cette fois-ci sur le polymorphisme pour appliquer le principe de *separation of concerns*. Dans les deux cas, on voit que l'interface créée par le polymorphisme consistue une abstration utile contribuant à la modularité et donc à la réemployabilité du code.

On distingue souvent trois formes principales de polymorphisme, les languages l'autorisant ne s'arrêtant souvent pas à une seule: 
* Le polymorphisme dit *ad hoc* dans lequel une fonction/méthode d'un nom donnée peut avoir différentes implémentation suivant le type de ses arguments. Dit autrement, le comportement de la fonction dépend du type de ou de ses arguments. On parle alors de surcharge (*overriding*) de fonctions/d'opérateurs. Ce type de polymorphisme se rencontre le plus souvent dans les languages statiquement typés. Suivant si la fonction à finalement appeler est déterminée à la compilation ou à l'exécution, on parle de *static* ou de *dynamic dispatch*.
* Le polymorphisme paramétrique dans lequel le type d'un ou plusieurs arguments d'une fonction/méthode n'est pas spécifié mais remplacé par un type abstrait pouvant en représenter une large variété. Ce type de polymorphise est notamment très présent en programmation fonctionnelle. On le rencontre aussi en POO via des *features* comme les *templates* (C++) ou les *generics* (Java, C#). Par définition, c'est une forme qu'on rencontre le plus souvent dans les languages statiquement typés. 
* Le sous-typage (*subtyping*) dans lequel on restreint (aux sous-types d'un type de base) la gamme de types pouvant être utilisés. Une fonction écrite pour un type T fonctionnera aussi avec des sous-types de T (respectant le principe de substitution de Liskov, cf. le "L" des SOLID *principles*). Le mécanisme définissant la relation de sous-typage dépend du système de types choisis (cf. systèmes nominatifs/structurels). En POO et que le systèmes de types soit nominatif, structurel ou *ducked-typed*, la relation de sous-typage correspond le plus souvent à une relation l'héritage: une sous-classe est un sous-type du type de sa classe parente.

## Principes de d'*object-oriented design*
De manière générale, on cherche à construire un code satisfaisant les objectifs suivants:
* Minimiser la duplication du code: *Don't repeat yourself* (DRY) *principle*
* Minimiser le couplage entre les composants ce qui permet:
    * D'obtenir un code modulaire et de maximiser le réemploi du code (et du même coup de minimiser sa duplication),
    * De minimiser les éventuels impacts indésirables lors de l'introduction de modifications ou d'évolutions: on maximimise ainsi la maintenabilité et l'évolutivité du code.
* Préserver l'intelligibilité d'ensemble du code, la lisibilité de sa logique d'ensemble. Cela passe par exemple par la maximisation de la cohérence des objets créés et l'emploi raisonné de *design patterns* dont certains peuvent rendre le code plus difficile à comprendre.

Construire un code, c'est finalement décider de la répartition de responsabilités entre les entités qui le composent. Tous les principes d'*object-oriented design* sont ainsi des principes d'assignations de responsabilité visant à contribuer à un ou plusieurs des objectifs précédemment cités.

On présente ici les trois grands principes ou familles de principes les plus connus:
* Les SOLID *principles* 
* La *Separation of Concerns* (SoC)
* Les *General Responsibility Assignment Software Patterns/Principles* (GRASP)

Remarque: On pourrait ajouter comme principe qu'on n'ajoute pas de fonctionnalité "au cas où". Si on présens cependant qu'un point du code aura à évoluer dans le futur, on peut se préoccuper de l'évolutivité future de ce point dès le design. 

### SOLID *principles*
Tous ces principes poussent à la modularité du code et à la minimisation des couplages entre ses différents composants. Tous impliquent potentiellement des restrictions à la programmation.

#### *Single-responsablity principle*
Une classe, fonction ou méthode ne doit avoir qu'une seule responsabilité, n'être responsable que d'un seul périmètre sur l'ensemble des fonctionnalités apportées par le software. Le principe d'unique responsabilité est entendu au sens qu'une entité respectant ce principe ne doit changer que pour une seule raison.

#### *Open/closed principle*
Une classe doit être ouverte à l'extension mais fermée à la modification : toute nouvelle fonctionnalité doit pouvoir être ajoutée sans modification du code existant. "Ouvert" signifie que la classe, son comportement peut être étendu, "fermé" signifie que tout changement ne peut se faire que par extension, sans modification du code source de la classe. En pratique, l'extension d'une classe respectant ce principe se fait par la création de nouvelles implémentations s'appuyant ainsi sur les notions d'interface (ce principe est donc à rapprocher du "D" des SOLID *principles*) et de polymorphisme.

L'*Open/closed principle* peut être implémenté par héritage: tout nouveau comportement est implémenté en sous-classant une classe parente qui de fait n'est pas modifiée. L'héritage n'est toutefois pas la seule façon d'imlémenter le principe, d'autant qu'il peut présenter des inconvénients. Le principe est ainsi parfois raffiné en *polymorphic Open/closed principle*, celui-ci s'appuyant sur des interfaces plutôt que sur des classes parentes pour autoriser différentes implémentations. Dans ce cas, ce sont les interfaces qui sont fermées aux modifications. Cela n'empêche pas l'héritage, s'il est bénéfique que deux implémentations d'une même interface, s'appuyer sur l'héritage ou l'association (les deux implémentation délèguent à un même objet).

#### Liskov *substitution principle*
Une instance d'un type T doit pouvoir être remplacé par une instance d'un sous-type de T sans que la cohérence du programme en soit modifiée. Cela signifie que toute classe doit pouvoir être remplacée par sa classe parente sans casser l'application et inversement. En particulier cela signifie notamment que les sous-classes:
* Ne peuvent pas se monter plus strictes que leur parente sur la validation des arguments d'une méthode: remplacer la classe parente par une de ses filles lèverait alors une exception.
* Ne peuvent pas se montrer moins strictes que leur parente sur le type retourné par une méthode. 

Le principe de substitution de Liskov impose des restrictions aux relations d'héritage et notamment une structure commune aux différentes classes. En général elle sont aussi des implémentations d'une ou de plusieurs interfaces, le principe impose qu'elles partagent au minimum la structure de celles-ci (*design by contracts* ?).

#### *Interface seggregation principle*
L'idée derrière ce principe est qu'une classe ne devrait pas dépendre, ne devrait pas être obligée d'implémenter des méthodes qu'elle n'utilise pas. Cela peut imposer de diviser des interfaces volumineuses en interfaces plus petites. Préférer plusieurs interfaces spécifiques à une seule interface générale. Cela ne mène pas nécéssairement à un morcellement des interfaces: en Java par exemple une interface peut en implémenter d'autres, on peut ainsi définir des hiréarchies d'interfaces et/ou en construire par combinaisons. 

#### *Dependency inversion principle*
Ce principe énonce plus précisement qu'un module de haut-niveau ne doit pas dépendre de modules de plus bas niveau. Tous ne doivent dépendre que d'abstractions (d'interfaces). Dit autrement: les abstractions (modules de haut-niveau) ne doivent pas dépendre d'implémentations (modules de bas-niveau). Toutefois l'inversion de dépendances ne signifie pas que les modules de plus bas niveau doivent à l'inverse dépendre de ceux de plus haut niveau. Le principe énonce également que les implémentations doivent dépendre d'abstractions. Ce principe inverse la perception que peuvent avoir les gens de la programmation orientée-objet, d'où le nom de *dependency inversion principle*. Une architecture "traditionnelle" voudrait en effet que des composants de bas-niveau soient consommés par des composants de plus haut-niveau pour la réalisation d'une tache. Cependant les second deviennent de fait dépendants, couplés aux premiers ce qui limite la généricité, la capacité de réemploi du code. 

Le principe d'inversion de dépendance vise à réduire le couplage entre composants de haut- et bas-niveau, entre une classe et ses dépendances en introduisant la médiation d'une couche d'abstraction, l'interface, augmentant du même coup la réutilisabilité et la capacité à tester isolément (notamment en mockant toutes les classes non-testées) tous les composants. La réduction du couplage entre les composant permet également de les isoler de changements de design survenant ailleurs.

Remarque: L'application de ce principe impose des restriction de programmation dont on peut citer:
* La création d'objet doit passer par un *creational pattern* afin d'éviter d'introduire une dépendance par rapport à l'objet créé.

### *Separation of Concerns* (SoC) *principle*
La *separation of concerns* est un *design principle* consistant à séparer un programme informatique en sections distinctes, chacune se préoccupant d'un aspect du programme. Une conséquence de l'application de ce principe est l'obtention d'un programme modulaire (qui peut par exemple se structurer en couches). Chaque préoccupation (*concern*) du programme se retrouve encapsulé dans une section du code présentant une interface. Un code modulaire procure davantage de degrés de liberté en termes d'évolutivité, de maintenance et de simplification. Cacher les implémentations derrières des interfaces permet en particulier de pouvoir développer ou changer des parties du code sans impacter les autres. L'utilisation d'interfaces permet de découpler deux composants qui sont dès lors davantage indépendants, et de travailler séparemment sur chacun d'entre eux.

Avertissement: Segmenter son code en modules spécialisés (par fonctionnalité et/ou par niveau de détail par exemple) est une bonne chose mais peut ne pas suffire. Il faut également se préoccuper des relations de dépendance entre les différentes parties du programme afin de ne pas se retrouver avec un code certes modulaire mais avec des dépendances inextricables.

Poussant à l'application du principe d'abstraction avec pour effet l'introduction d'interfaces, le *separation of concerns principle* peut se voir comme intégrant au moins les S et D des SOLID *principles*.

Dans les fait, l'implémentation de ce principe dépend des possiblités données par le language. Dans les languages orientés-objet, chaque *concern* est encapsulé dans un objet. Il peut l'être égalment dans des fonctions (on parle aussi parfois de *procedures* ou de *routines*) elles-mêmes rassemblées par module. L'application du SoC participe de la logique qui guide l'ensemble des *designs patterns* qu'il soient propres à un paradigme ou attachés à l'architecture. 

###  General Responsibility Assignment Software Patterns/Principles (GRASP)
Les GRASP sont un autre ensemble de règles et de principes de POO moins connus que les SOLID *principles* et davantage focalisés sur l'assignation des responsabilités entre différents composants sans faire appel à des *design patterns*. 

Avant d'aborder les principes eux-mêmes se pose la question de la définition de la responsabilité d'un bloc de code. On peut donner une définition simple: une responsabilité est soit l'obligation d'accomplir une tache (un comportement) ou détenir de l'information (savoir). Elle peut être assumée par un objet ou différents objets en collaboration.

Parmi les différentes taches on peut citer créer un objet, traiter des données, réaliser un calcul, initialiser des objets, coordonner les actions entre objets, etc. De l'autre côté, on entend par "savoir" le fait de gérer des données, privées ou publiques.

Les GRASP sont au nombre de 9:
1. *Information Expert*
2. *Creator*
3. *Controller*
4. *Low Coupling*
5. *High Cohesion*
6. *Indirection*
7. *Polymorphism*
8. *Pure Fabrication*
9. *Protected Variations*

#### *Information expert*
Ce principe veut répondre à la question: "Sur quel principe de base s'appuyer pour assigner des responsabilités à des objets ?". 

L'*information expert principle* énonce: "On a assigne une responsabilité à la classe possédant l'information lui permettant de l'assumer".

#### *Creator*
Ce principe veut répondre à la question: "Quel classe doit assumer la responsabilité de créer l'objet A ?"

Le *creator principle* énonce: "Assigner à une classe B la responsabilité de créer A si au moins une de ces assertions est vraie:"
* B est composé de/aggrège A,
* B comptabilise A,
* B utilise A de près,
* B possède les données nécessaires à l'initialisation de A.

Ce principe est à rapprocher des différents *creational design patterns*.

#### *Controller*
Ce principe vise un cas d'usage un peu plus précis dans lequel notre application présente une couche comme une UI chargée de capturer des évenéments. Comment assigner la responsabilité du traitement de l'événement provenant de la couche d'UI (requête) ? Les logiques à exécuter sont souvent rassemblée dans une autre couche (*domain layer*) et la prise en charge d'une requête peut exiger une l'exécution d'une combinaison complexes de logiques. Un *controller object* est le premier objet à recevoir la requête et la satisfait en coordonnant les objets nécessaires de la *domain layer*. Le *controller object* ne réalise que peut de travail lui même, il en délègue l'essentiel à la *domain layer*. Il ne fait que rassembler la logique de haut niveau permettant de satisfaire un type de requête (la classe *controller* pouvant connaître différentes implémentations). En général, les couches sont déjà bien séparées et des objets se chargent du traitement des requête. Le *controller principle* sert à identifier lesquels sont réellement en charge du contrôle. On les qualifiera alors explicitement de *controllers* et les refactorisera éventuellement afin qu'ils délèguent autant que nécessaire à la *domain layer*.

Les *design patterns* Mediator et Facade viennent répondre à des problématiques semblables. On retrouve également le principe de la séparation des responsabilités entre Director et Builder dans le *pattern* Builder. 

#### *Low coupling*
Le couplage est une mesure du point auquel un élément est dépendant d'un autre. Assurer un faible couplage entre les objets, c'est les rendre plus indépendants et plus isolés des changements pouvant intervenir ailleurs. Ce principe est l'objectif de nombreux autres comme les SOLID *principles* ou l'utilisation d'abstractions. 

Ce principe nous pousse à répondre à la question: "Comment réduire l'impact d'un changement ? Comment réduire les dépendances entre objets et augmenter la réemployabilité du code ?". Le *low coupling principle* est davantage un principe d'évaluation permettant de juger de différentes alternatives: on garde l'assignation de responsabilités produisant le couplage le plus faible.

#### *High cohesion*
Comme le *low coupling*, le *high cohesion principle* est un principe d'évaluation d'alternatives. La cohésion s'intéresse à la cohérence des responsabilités assignées, rassemblées au sein d'une même entité. On cherche ainsi à se prémunir contre les objets rassemblant des comportements ou des données ayant peu ou rien à voir entre deux. 

Ce principe vient répondre à la question: "Comment avoir des objets dédiés à un ensemble cohérent de taches ?" Des objets cohérents auront aussi l'avantage d'être plus facilement compréhensibles et maintenables. On ne le sépare en général pas du *low coupling principle*: on recherche l'assignation de responsabilités permettant d'obtenir les objets les plus découplés et cohérents possibles. 

#### *Indirection*
L'*indirection principle* veut répondre à la question: "Comment assigner les responsabilités de façon à ne pas avoir de couplage direct entre deux entités ?". La solution proposée est d'utiliser un objet intermédiaire pour réaliser la médiation entre les deux entités qui ne sont alors plus directement couplées et qui n'interagissent d'indirectement. Le *design pattern* Mediator vient notamment répondre à une telle problématique. 

L'utilisation d'un tel objet indirect peut toutefois se discuter puisqu'elle rend le code plus difficile à appréhender: la classe à qui est confiée l'exécution d'une tache n'est plus visible depuis le code du *controller*/*director*.

#### *Polymorphism*
Ce principe vient répondre à la question: "Comment gérer les situations où les différentes alternatives possibles dépendent du type de l'objet ?". La solution consiste à s'appuyer sur des opérations polymorphiques (la responsabilité est finalement assignée au type). On peut par exemple refactoriser le code client de façon à ce qu'il s'appuie sur une interface dont les différentes alternatives sont les implémentations (polymorphisme par sous-typage).

De nombreux *design patterns* s'appuient sur ce principe (ex: Strategy). 

#### *Pure Fabrication*
Il arrive parfois que certaines responsabilités soient difficiles à assigner à des objets tout en maintenant les *high cohesion* et *low coupling* principles. L'idée du *pure fabrication principle* est de ne pas trop se contraindre à vouloir donner une signification métier (*domain*) à tous les objets créés et qu'il faut mieux dans certains cas se créer un objet *ad hoc*, artificiel (ce à quoi fait allusion le terme *pure fabrication*) mais fortement cohérent.

#### *Protected Variations*
Ce principe vient répondre à la question: "Comment concevoir les éléments d'un programme de façon à ce que des modifications dans l'un n'ai pas ou peu d'impact indésirable sur les autres (qui sont donc "protégés des variations des autres") ?" La réponse apportée préconise d'identifier ces points d'instabilités ou dont on peut prédire qu'ils auront à évoluer et réassigner les responsabilités de façon à construire une interface stable autour d'eux. Ce principe est très lié aux *low coupling* et *polymorphism principles*. L'objectif visé par ce principe est l'objet de bien d'autres recommendations de design et de *design patterns*. C'est notamment l'un des objectifs poursuivi par quasiment tous les SOLID *principles*. 

Comme pour les *low coupling* et *high cohesion principles*, on peut voir le *protected variations principle* comme un principe d'évaluation: on cherche l'assignation de responsabilités permettant d'obtenir la meilleur évolutivité (*ease of change*) possible. 

## *Inversion of Control* (IoC) & *Dependency injection* (DI)
L'inversion de contrôle est un principe de programmation (également rangé dans la catégorie des *architectural design patterns*) dans lequel le *flow of control* (ordre dans lequel les instructions d'un language impératif sont exécutées) "traditionnel" est inversé. Dans un programme écrit intuitivement, sans préoccupation de design, c'est le code spécialisé (*custom*), celui qui exprime les visées de l'ensemble du programme, qui appelle du code de librairies spécialisées pour l'exécution de taches relativement génériques. En IoC, les portions spécialisées du programme reçoivent le *flow of control* d'un *framework* générique. C'est le *framework* générique qui appelle le code spécialisé et plus l'inverse. Le contrôle a été passé du code spécialisé au *framework*. Le *framework* manipule des concepts de haut niveau de façon à pouvoir en maximiser son réemploi. Suivant le cas d'usage, chacun de ces concepts recevra une implémentation différente, chacune pouvant dès lors être développée indépendemment. 

Remarque: On parle parfois du principe de l'IoC comme du *Hollywood principle* pour illustrer que ce n'est plus le code spécialisé qui appelle du code générique mais l'inverse: *Don't call us, we'll call you*.

Le but de l'IoC est d'augmenter la testabilité et la modularité du code, le rendant du même coup plus facilement extensible. Ce principe est lié mais distinct du *dependency inversion principle* qui décrit une modalité d'implémentation de l'IoC (dépendre d'une interface et non d'implémentations). 

L'IoC est un principe de design dans lequel on découple une logique de haut niveau des implémentations concrètes de chacune des étapes de plus bas niveau la composant. Les impémentations concrètes des logiques de plus bas niveau deviennent des dépendances de l'objet encapsulant la logique de plus haut niveau. Avec L'application de ce principe viennent différents avantages:
* La logique de haut niveau devient en un sens configurable : elle est donc réemployable et d'utilisation flexible.
* La logique de haut niveau devient plus facilement testable (demande également qu'on puisse facilement lui passes ses dépendances - cf. *dependency injection*) : il suffit de mocker ses dépendances. 
* Le principe nous pousse à prêter attention à l'assignation des responsabilités entre la logique de haut niveau et ses dépendances, améliorant du mêmes coup la cohérence des unités du programme.

Dans la pratique, ce principe général connait une large variété d'implémentations. 

Les *design patterns* Strategy et Template contribuent tous deux à mettre en pratique le principe d'IoC:
* Dans le Template la logique de haut niveau est factorisée dans la *template method* et les logiques de plus bas niveau sont regroupées dans les autres méthodes de la *template class*. Le Template fonctionne par héritage: chaque implémentation de la *template class* vient réimplémenter tout ou parties des logiques de plus bas niveau d'une *base template class*. L'implémentation désirée de la *template class* est choisie à l'exécution.
* Le *pattern* Strategy se repose sur l'association. Une *strategy* est en général une dépendance d'un objet contenant la logique de plus haut niveau, ce dernier déléguant l'exécution d'une partie de celle-ci à la *strategy*. L'utilisation de la composition permet une plus grande modularité qu'avec un *template object* : on peut choisir indépendemment (et à l'exécution) les implémentations de chaque élément de la logiques de haut niveau (eux-mêmes implémentant le *pattern* Strategy) alors qu'avec un Template, à un jeu de logiques de bas niveau doit correspondre une implémentation de la *base template class*. Une implémentation de Template rassemble a priori des méthodes compatibles entre elles ou formant au moins un ensemble cohérent (un peu comme dans une Abstract Factory). Dans le Template, il semble sous-entendu qu'une modularité complète n'est pas souhaitable, il est important que le programmeur ne perde pas les autres méthodes de vues lorsqu'il développe une nouvelle logique de bas niveau.

Le deuxième point fait apparaître clairement une relation de dépendance entre l'objet encapsulant la logique de haut niveau et les différentes stratégies le composant. Comment lui passer les instances des différentes stratégies, ses dépendances ?
* Une première solution serait l'utilisation d'une *factory method* ou d'une Abstract Factory. Dans le second cas, on a fait que déplacer le problème: il ne s'agit plus de passer à l'objet la strategie elle-même mais l'implémentation de *factory* lui permettant de la récupérer.
* Une deuxième solution consiste à injecter ses dépendances à l'objet, une telle injection peut prendre différentes formes:
    * La dépendance est injectée à l'objet via son constructeur (*constructor injection*).
    * La dépendance est directement settée par l'utilisation directe d'une *setter method* (*setter injection*). Dans les languages dynamiques comme Python, cela peut se réaliser par une expression de la forme `obj.attribute = value`. Les accusations de *monkey-patching* ne sont pas très loin.
    * La dépendance est injectée par une méthode dédiée faisant partie de l'interface publique de l'objet (*interface injection*).
* Une troisième solution consiste à s'appuyer sur un registre appelé *service locator* dans lequel sont rassemblées les dépendances qui sont ensuite récupérables via une méthode statique. On peut distinguer deux types de registres:
    * Le *configuration registry* qui n'est initialisé qu'une fois à l'aide de configurations extérieures.
    * Le *subscriber registry* qui est dynamique et peut être peuplé par différentes entités du programme. Potentiellement implémenté à l'aide du *design pattern* Observer où le registre joue le rôle du *Subject* observé.

On a supposé implicitement jusqu'ici que les différentes dépendances étaient déjà instanciées et initialisées, l'objet dépendant ne faisant que les récupérer. Ce code là est typiquement refactorisé dans objets dédiés (nommés *providers* dans les *frameworks* de *dependency injection* (DI)) au niveau desquels on retrouve tout le panel des *creational patterns*. Cela présuppose également l'existence d'un code appelé injecteur (*injector*) qui utilise ces objets pour créer les dépendances et les injecter à l'objet les nécessitant.

On remarque qu'au delà d'être un principe de séparation des logiques de haut et bas niveau, l'IoC pousse à dissocier comportements et construction.

Commençons par préciser qu'on n'a pas besoin de *frameworks* de DI pour faire de la *dependency injection*/IoC. Dans la terminologie de ces derniers, un objet et ses dépendances sont respectivement appelés *client* et *services*. On trouve dans tous les *frameworks* un objet (ou un ensemble d'objets fonctionnant de concert) jouant le rôle d'injecteur (*injector*), ce dernier ayant pour responsabilité de construire les dépendances du client et de les lui injecter (et voire de construire le client lui même). Ce ou ces objets injecteurs voient leur dénomination varier suivant les *frameworks* : *provider* (qui en général définissent la stratégie de création d'une dépendance particulière), *container* (parfois défini comme une collection de *providers*), *assembler*, *construction code*, *main*, etc. Un des avantages de ces *frameworks* est qu'ils permettent de gérer des arbres de dépendances complexes (*object trees*) qui peuvent sinon rendre un code rapidement illisible.

Il semble que les *frameworks* de DI se recontrent moins ou sont en tout cas plus légers dans les languages à typage dynamique comme Python (pour lequel on mentionne [`dependency_injector`](http://python-dependency-injector.ets-labs.org/index.html), [`serum`](https://github.com/suned/serum) ou encore [`dpy`](https://github.com/google/dpy)). Il semble par exemple que la *dependency injection* facilite fortement les tests dans les languages statiquement typés (là où un language dynamique pourrait recourir au *monkey-patching* (?)). Question à approfondir.

Configurations:
Convention (code/default values dans le code qu'on peut éventuellement surcharger) over configurations (config file) principle
Injection de confs
* CLI: ce qui peut changer d'un lancement à l'autre
* Config file: ce qui ne doit pas changer, paramétrage
* Variable d'env
Confs avec le code:
* convention over configurations
* settings.py ou config.py ou fichier externe (ini, yaml, etc. ?). 
Confs vs constantes (constants.py): les constantes ne sont que du refactoring
A l'intérieur du programme et pour la création d'objets, les confs sont finalement des dépendences comme les autres. Comment les mettre à disposition ? Registry ? les injecter ? Devoir les faire descendre profondément n'est-il pas un signe qu'on a un dependency graph compliqué ? Ne sont-elles pas un des cas où la DI par service locator est particulièrement indiquée ?

Une conf c'est finalement une dépendance, comment on l'injecte-t-on ? 

Remarque: registry statique: on doit s'assurer qu'il est initialisé. Notamment problématique si on met à disposition un package en plus de l'application qui s'appuie dessus: pour l'app, le main s'assure de l'initialisation du registre, dans le package c'est moins sûr.

https://www.hackerearth.com/fr/practice/notes/samarthbhargav/a-design-pattern-for-configuration-management-in-python/
https://stackoverflow.com/questions/6198372/most-pythonic-way-to-provide-global-configuration-variables-in-config-py
https://stackoverflow.com/questions/5710758/python-pattern-for-sharing-configuration-throughout-application
https://stackoverflow.com/questions/1314730/which-design-patterns-can-be-applied-to-the-configuration-settings-problem
https://hackernoon.com/4-ways-to-manage-the-configuration-in-python-4623049e841b